In [1]:
import requests
import pandas as pd
import numpy as np
import signal
import time

In [2]:
API_KEY = {'X-API-key': 'RPCJTKLW'}

In [3]:
shutdown = False

class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error. Please check API key.')

def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Authorization error. Please check API key.')

def buy(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'BUY'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'BUY', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def sell(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'SELL'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'SELL', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def cancel(session, order_id):
    order_id = order_id # assuming the order to cancel has ID 100
    resp = session.delete('http://localhost:9999/v1/orders/{}'.format(order_id))
    if resp.ok:
        status = resp.json()
        success = status['success']
        print('The order was successfully cancelled?', success)
        
def tender_list(session):
    resp = session.get('http://localhost:9999/v1/tenders')
    if resp.ok:
        lst = resp.json()
        if lst:
            return lst[0]
        else:
            return None
        
def acc_tender(session, tender, price = None):
    if tender['is_fixed_bid'] is True:
        resp = session.post('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'id': tender['tender_id']})
        if resp.ok:
            print('tender accepted!')
    else:
        resp = session.post('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'price': price, 'id': tender['tender_id']})
        if resp.ok:
            print('tender accepted!')
      
    
def dec_tender(session, tender):
    resp = session.delete('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'id': tender['tender_id']})
    if resp.ok:
        print('tender declined!')
        
def current_price(session, ticker):
    resp = s.get('http://localhost:9999/v1/securities', params = {'ticker': ticker})
    json = resp.json()
    return json[0]['last']


def buy_mult(session, ticker, type, mult):
    if ticker == 'RUTC':
        for i in range(mult):
            buy(session, ticker, type, 10000)
    else:
        for i in range(mult):
            buy(session, ticker, type, 5000)
        
def sell_mult(session, ticker, type, mult):
    if ticker == 'RUTC':
        for i in range(mult):
            sell(session, ticker, type, 10000)
    else:
        for i in range(mult):
            sell(session, ticker, type, 5000)
        

def close_pos(session, ticker, position):
    if ticker == 'RUTC':
        if position < -10000:
            mod = np.abs(position) // 10000
            buy_mult(session, ticker, 'MARKET', mod)
            buy(session, ticker, 'MARKET', position - (mod * 10000))
        elif -10000 < position < 0:
            buy(session, ticker, 'MARKET', np.abs(position))
        elif 0 < position < 10000:
            sell(session, ticker, 'MARKET', position)
        elif position > 10000:
            mod = np.abs(position) // 10000
            sell_mult(session, ticker, 'MARKET', mod)
            sell(session, ticker, 'MARKET', position - (mod * 10000))
        else:
            return
    else:
        if position < -5000:
            mod = np.abs(position) // 5000
            buy_mult(session, ticker, 'MARKET', mod)
            buy(session, ticker, 'MARKET', position - (mod * 5000))
        elif -5000 < position < 0:
            buy(session, ticker, 'MARKET', np.abs(position))
        elif 0 < position < 5000:
            sell(session, ticker, 'MARKET', position)
        elif position > 5000:
            mod = np.abs(position) // 5000
            sell_mult(session, ticker, 'MARKET', mod)
            sell(session, ticker, 'MARKET', position - (mod * 5000))
        else:
            return
        

In [4]:
 with requests.Session() as s:
        s.headers.update(API_KEY)
        tick = get_tick(s) 
        resp_gear_bull = s.get('http://localhost:9999/v1/securities', params = {'ticker' : 'BULL'})
        bull_pos = int(resp_gear_bull.json()[0]['position'])
        resp_gear_bear = s.get('http://localhost:9999/v1/securities', params = {'ticker' : 'BEAR'})
        bear_pos = int(resp_gear_bear.json()[0]['position'])
        
        print(bull_pos)
        close_pos(s, 'BULL', bull_pos)
        close_pos(s, 'BEAR', bear_pos)       

0


In [9]:
 with requests.Session() as s:
        s.headers.update(API_KEY)
        tick = get_tick(s) 
        resp_gear_rutc = s.get('http://localhost:9999/v1/securities', params = {'ticker' : 'RUTC'})
        rutc_pos = int(resp_gear_rutc.json()[0]['position'])

        print(rutc_pos)
        close_pos(s, 'RUTC', rutc_pos)

-7000
The market buy order was submitted and has ID 2414
